## 1. Data source: PAMAP2 (UCI)

**Dataset:** *PAMAP2 Physical Activity Monitoring* (UCI Machine Learning Repository)

- **UCI dataset page (download + description):** https://archive.ics.uci.edu/dataset/231/pamap2+physical+activity+monitoring  
- **Local path (expected):** download and unzip the dataset into `./data/pamap2/` (the `data/` directory is not committed to git)  
- **Dataset readme (format / notes):** https://archive.ics.uci.edu/ml/machine-learning-databases/00231/readme.pdf  
- **License:** Creative Commons Attribution 4.0 International (**CC BY 4.0**)

PAMAP2 is a public wearable-sensing dataset for **human activity recognition (HAR)**. It contains recordings from **9 subjects** performing **18 labeled activities** (plus label `0` for transient/other) while wearing three IMU sensors (**hand/wrist**, **chest**, **ankle**) and a heart-rate monitor.

**File format (from the dataset readme):** each row has **54 columns**:
- `1` timestamp (s)
- `2` activityID
- `3` heart rate (bpm)
- `4–20` IMU hand (wrist)
- `21–37` IMU chest
- `38–54` IMU ankle

Each IMU block contains:
- temperature (°C)
- 3D accelerometer (±16g) and 3D accelerometer (±6g)
- 3D gyroscope (rad/s)
- 3D magnetometer (μT)
- orientation (not valid in this collection)

**Activity IDs (18 activities + transient):**
1 lying; 2 sitting; 3 standing; 4 walking; 5 running; 6 cycling; 7 Nordic walking;  
9 watching TV; 10 computer work; 11 car driving; 12 ascending stairs; 13 descending stairs;  
16 vacuum cleaning; 17 ironing; 18 folding laundry; 19 house cleaning; 20 playing soccer; 24 rope jumping;  
0 other (transient activities)

**What we use in this notebook:** **Protocol** recordings only, with a **watch-like setup**: wrist IMU signals (ACC/GYRO/MAG) as inputs and the **activityID** as the multi-class label. We **exclude label `0`** (“transient/other”) to keep the benchmark focused on the defined activities.
